# Topic modelling Q&D pilot
https://medium.datadriveninvestor.com/nlp-with-lda-analyzing-topics-in-the-enron-email-dataset-20326b7ae36f
https://github.com/shoreason/enron-topic-modeling/blob/master/enron_lda.ipynb


In [1]:
import pandas as pd
import email


In [2]:
# # Create a small set of emails (10 000)
# emails_read_all = pd.read_csv("data/emails.csv")
# enron = emails_read_all.head(10000)
# enron.to_csv('data/emails_df_10000.csv', index=False)

In [3]:
enron = pd.read_csv('data/emails_df_10000.csv')
#enron = pd.read_csv('./csv/emails_df.csv')

In [4]:
enron.shape #10 000 rows, 18 columns
print(enron.columns)
enron.rename(columns={'content': 'body'}, inplace=True)


Index(['file', 'message'], dtype='object')


In [5]:
def get_text_from_email(msg: email.message.Message) -> str:
    """To get the content from email objects
    :param msg: the email object
    :return: the content of the email
    """
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line: str) -> frozenset:
    """
    To separate multiple email addresses
    :param line: the line of the csv file
    :return: a set of email addresses
    """
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [6]:

# Parse the emails into a list email objects
messages = list(map(email.message_from_string, enron['message']))
enron.drop('message', axis=1, inplace=True)

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    enron[key] = [doc[key] for doc in messages]

# Parse content from emails
enron['content'] = list(map(get_text_from_email, messages))

# Split multiple email addresses
enron['From'] = enron['From'].map(split_email_addresses)
enron['To'] = enron['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
enron['user'] = enron['file'].map(lambda x:x.split('/')[0])
del messages

In [7]:
enron

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,bass-e/_sent_mail/1047.,<21376194.1075854731510.JavaMail.evans@thyme>,"Tue, 25 Jan 2000 02:31:00 -0800 (PST)",(eric.bass@enron.com),(brian_hoskins@enron.com),Re: Evite: Super Bowl Party,1.0,text/plain; charset=us-ascii,7bit,Eric Bass,"""Brian Hoskins"" <Brian_Hoskins@enron.com>",,,\Eric_Bass_Dec2000\Notes Folders\'sent mail,Bass-E,ebass.nsf,"shes pretty sexy, huh? are we getting togethe...",bass-e
9996,bass-e/_sent_mail/1048.,<2401643.1075854731531.JavaMail.evans@thyme>,"Tue, 25 Jan 2000 02:15:00 -0800 (PST)",(eric.bass@enron.com),(danielles@jonesgranger.com),,1.0,text/plain; charset=us-ascii,7bit,Eric Bass,danielles@jonesgranger.com,,,\Eric_Bass_Dec2000\Notes Folders\'sent mail,Bass-E,ebass.nsf,i copied your idea - and it screwed up your name!,bass-e
9997,bass-e/_sent_mail/1049.,<32002412.1075854731553.JavaMail.evans@thyme>,"Tue, 25 Jan 2000 01:36:00 -0800 (PST)",(eric.bass@enron.com),(david.baumbach@enron.com),9912 Texas Financial Liquidations,1.0,text/plain; charset=us-ascii,7bit,Eric Bass,David Baumbach,,,\Eric_Bass_Dec2000\Notes Folders\'sent mail,Bass-E,ebass.nsf,---------------------- Forwarded by Eric Bass/...,bass-e
9998,bass-e/_sent_mail/105.,<21429983.1075854709202.JavaMail.evans@thyme>,"Wed, 6 Dec 2000 06:03:00 -0800 (PST)",(eric.bass@enron.com),(lwbthemarine@bigplanet.com),,1.0,text/plain; charset=us-ascii,7bit,Eric Bass,lwbthemarine@bigplanet.com,,,\Eric_Bass_Dec2000\Notes Folders\'sent mail,Bass-E,ebass.nsf,did you buy any enron in the 60s?,bass-e


In [8]:
enron = enron[['From', 'To', 'Subject', 'content']]
print(enron.head())



                        From                         To    Subject  \
0  (phillip.allen@enron.com)     (tim.belden@enron.com)              
1  (phillip.allen@enron.com)  (john.lavorato@enron.com)        Re:   
2  (phillip.allen@enron.com)   (leah.arsdall@enron.com)   Re: test   
3  (phillip.allen@enron.com)    (randall.gay@enron.com)              
4  (phillip.allen@enron.com)     (greg.piper@enron.com)  Re: Hello   

                                             content  
0                          Here is our forecast\n\n   
1  Traveling to have a business meeting takes the...  
2                     test successful.  way to go!!!  
3  Randy,\n\n Can you send me a schedule of the s...  
4                Let's shoot for Tuesday at 11:45.    


In [9]:
import re
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy # for lemmatization
# for plotting
import pyLDAvis
import pyLDAvis.gensim_models  # the module 'gensim' has renamed to gensim_models
import matplotlib.pyplot as plt

c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\joblib\backports.py:7: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [10]:
import nltk

# nltk.download('stopwords')

In [11]:
# prep NLTK stop words
from nltk.corpus import stopwords

# You have to install the word on your local machine , 3.84 GB!!
# https://www.nltk.org/data.html
stop_words = stopwords.words('english') # will cause error if not installed accordint to the instructions above
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'send', 'com']) # if replied the mail contains thes words -> remove them



In [18]:
print(enron.iloc[1]['content']) # displays info below

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  

My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.



In [21]:
# Convert email content to list
data = enron.content.values.tolist()
#data = enron.Subject.values.tolist()


In [22]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [23]:
data_words = list(sent_to_words(data))

In [25]:
print(data_words[1])

['traveling', 'to', 'have', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'trip', 'without', 'any', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'trip', 'is', 'even', 'desired', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meetings', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', 'too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', 'the', 'meetings', 'might', 'be', 'better', 'if', 'held', 'in', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go', 'is'

In [26]:
from gensim.models.phrases import Phrases, Phraser

In [27]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [28]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [29]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['forwarded', 'by', 'phillip', 'allen', 'hou', 'ect', 'on', 'pm', 'from', 'julie_gomez', 'am', 'to', 'hunter_shively', 'hou', 'ect', 'ect', 'phillip', 'allen', 'hou', 'ect', 'ect', 'cc', 'subject', 'alliance_netback_worksheet_hello', 'men', 'have', 'attached', 'my', 'worksheet', 'in', 'case', 'you', 'want', 'to', 'review', 'the', 'data', 'while', 'am', 'on', 'holiday', 'thanks', 'julie']


In [30]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [31]:
# Remove Stop Words 
data_words_nostops = remove_stopwords(data_words)

In [32]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# python -m spacy download en

In [39]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# nlp = spacy.load('en', disable=['parser', 'ner']) # this is how it used to be
nlp = spacy.load('en_core_web_sm')


c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\catalogue\__init__.py:135: DeprecationW

In [40]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[200])

['forward', 'phillip', 'hunter_shively', 'ect', 'ect', 'man', 'attach', 'case', 'want', 'review', 'datum', 'holiday', 'thank']


In [41]:
print(data_lemmatized[1])

['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']


In [42]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [43]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [44]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [57]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [46]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.013*"send" + 0.013*"message" + 0.011*"thank" + 0.011*"original" + 0.007*"time" + 0.007*"mail" + 0.007*"information" + 0.007*"know" + 0.006*"call" + 0.006*"right"'), (1, '0.078*"font" + 0.068*"ect" + 0.067*"size" + 0.062*"align" + 0.031*"ee" + 0.025*"hou" + 0.021*"tr" + 0.019*"right" + 0.016*"span" + 0.013*"http"'), (2, '0.019*"market" + 0.015*"iso" + 0.013*"price" + 0.012*"power" + 0.011*"energy" + 0.010*"say" + 0.010*"company" + 0.006*"state" + 0.006*"gas" + 0.005*"demand"')]


In [47]:
doc_lda = lda_model[corpus]

In [49]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.932341085729957


In [50]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5066800353277691


In [58]:
# Compute Coherence Score (num topics = 10)
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence value with 10 topics : ', coherence_lda)

In [51]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

c:\Users\vpala\Artificial Intelligence\BeCode\Clustering-Enron_UsedCase\enron_env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [52]:
pyLDAvis.display(vis)